# vLLM Chat

vLLM can be deployed as a server that mimics the OpenAI API protocol. This allows vLLM to be used as a drop-in replacement for applications using OpenAI API. This server can be queried in the same format as OpenAI API.

This notebook covers how to get started with vLLM chat models using langchain's `ChatOpenAI` **as it is**, as well as how to do structured generation with `ChatVLLMOpenAI`.

We assume you already have a vLLM server running. See the [vLLM README](https://docs.vllm.ai/en/latest/serving/deploying_with_docker.html) for instructions on how to deploy a vLLM server.

In [1]:
from langchain_community.chat_models import ChatVLLMOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

In [2]:
# The URL of the vLLM inference server
inference_server_url = "http://localhost:8000/v1"

llm = ChatVLLMOpenAI(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    openai_api_base=inference_server_url,
    temperature=0,
)

In [3]:
messages = [
    SystemMessage(
        content="You are a helpful assistant that translates English to Italian. Respond with only the translation."
    ),
    HumanMessage(
        content="Translate the following sentence from English to Italian: I love programming."
    ),
]
llm.invoke(messages).content

'Mi piace programmazione.'

You can make use of templating by using a `ChatPromptTemplate`.

In [4]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            "You are a helpful assistant that translates {input_language} to {output_language}. Respond with only the translation."
        ),
        HumanMessagePromptTemplate.from_template("{input}"),
    ]
)
chain = chat_prompt | llm | StrOutputParser()
print(
    chain.invoke(
        {
            "input_language": "English",
            "output_language": "Italian",
            "input": "I love programming.",
        }
    )
)
print(
    chain.invoke(
        {
            "input_language": "English",
            "output_language": "French",
            "input": "I love programming.",
        }
    )
)

Mi piace il programmazione.
J'adore le programmation.


# Structured Output With Guided Generation

The `ChatVLLMOpenAI` class supports the `with_structured_output` method for structuring the output of the model.
This is achieved through vLLM' support for guided generation.
This is a very powerful feature that allows even small models to accurately follow instructions.
Below we show several examples of how to use this feature.

We can use ensure that the model output can be parsed as a pydantic model

In [5]:
from typing import Literal

from langchain_core.pydantic_v1 import BaseModel, Field


# Example pydantic model
class CityModel(BaseModel):
    name: str = Field(..., description="Name of the city")
    population: int = Field(
        ..., description="Population of the city measured in number of inhabitants"
    )
    country: str = Field(..., description="Country of the city")
    population_category: Literal[">1M", "<1M"] = Field(
        ..., description="Population category of the city"
    )


structured_llm = llm.with_structured_output(CityModel, method="function_calling")

city_model = structured_llm.invoke("What is the capital of France?")
assert isinstance(city_model, CityModel)
print(city_model)

name='Paris' population=2141000 country='France' population_category='>1M'


vLLM has more extensive structuring methods.
For example, we can ensure that the model output is one of a set of options

In [6]:
allowed_choices = ["positive", "negative"]
structured_llm = llm.with_structured_output(
    allowed_choices,
    method="guided_choice",
)

print(structured_llm.invoke("I loved this movie!"))

positive


We can use regex too. For example to ensure that the output is a valid hex color code.

In [7]:
regex = r"#[0-9a-fA-F]{6}"
structured_llm = llm.with_structured_output(
    regex,
    method="guided_regex",
)

print(structured_llm.invoke("Give me the code for red."))

#FF0000


vLLM even supports EBNF grammars. Such as this example that restricts output to simple mathematical equations.

In [8]:
grammar = """?start: expression

?expression: term (("+" | "-") term)*

?term: factor (("*" | "/") factor)*

?factor: NUMBER
        | "-" factor
        | "(" expression ")"

%import common.NUMBER"""

structured_llm = llm.with_structured_output(
    grammar,
    method="guided_grammar",
)
print(
    structured_llm.invoke(
        "Translate two hundred and fifty-six minus three hundred and twenty-four divided by three into a mathematical expression."
    )
)

256-324/3


If you just want to ensure the output is valid JSON, you can select "json_mode" and specify `None` for the `schema`.

In [9]:
structured_llm = llm.with_structured_output(None, method="json_mode")
print(structured_llm.invoke("What is the capital of France?"))

{'data': 'Paris'}


In [11]:
structured_llm = llm.with_structured_output(None, method="guided_json")
print(structured_llm.invoke("What is the capital of France?"))

{'properties': {'capital': {'type': 'string'}}, 'required': ['capital']}


There is also support for `bind_tools`.

In [10]:
import json

tool_llm = llm.bind_tools([CityModel], tool_choice="CityModel")
res = tool_llm.invoke("What is the capital of France?")
json.loads(res.json())

{'content': '',
 'additional_kwargs': {'tool_calls': [{'id': 'chatcmpl-tool-71a325323dda4660ba88f9fd2105ee36',
    'function': {'arguments': '{ "name": "Paris", "population": 2140000, "country": "France", "population_category": ">1M" }',
     'name': 'CityModel'},
    'type': 'function'}]},
 'response_metadata': {'token_usage': {'completion_tokens': 33,
   'prompt_tokens': 17,
   'total_tokens': 50},
  'model_name': 'meta-llama/Meta-Llama-3-8B-Instruct',
  'system_fingerprint': None,
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-328ed011-383d-48fe-92a2-b1da67e5ed45-0',
 'example': False,
 'tool_calls': [{'name': 'CityModel',
   'args': {'name': 'Paris',
    'population': 2140000,
    'country': 'France',
    'population_category': '>1M'},
   'id': 'chatcmpl-tool-71a325323dda4660ba88f9fd2105ee36'}],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 17,
  'output_tokens': 33,
  'total_tokens': 50}}